### Low Luminosity Type IIP Supernovae Sample

Sample from [Das et al. 2025](https://arxiv.org/abs/2506.20068).

To familiarize myself with ZTF brokers, I will attempt to pull transient names, RA, and DEC from a broker. But I can also just check with the table provided from the first paper in the Das low luminosity Type IIP SN paper series.

In [21]:
import pandas as pd
import numpy as np
from alerce.core import Alerce
from astropy.coordinates import SkyCoord
import astropy.units as u
import psycopg

In [14]:
df_1 = pd.read_csv('data/zenodo_metasample.csv').drop_duplicates()
df_2 = pd.read_csv('data/TableA_full_machine_readable_params.csv').drop_duplicates()
print(df_1.columns, len(df_1))
print(df_2.columns, len(df_2))

Index(['name', 'firstdet', 'firstdetmag', 'jump', 'lastnondet', 'texp', 'gap',
       'Npre_g', 'Npre_r', 'Npost_g', 'Npost_r', 'slope_peak_r',
       'slope_peak_g', 'slope_peak_r10', 'slope_peak_g10', 'peakr', 'peakre',
       'peakg', 'peakge', 'peakrmjd', 'peakgmjd', 'redshift', 'peakr_app',
       'peakg_app', 'mean_EW', 'AV_NaD', 'colorexcess', 'Avhost', 'peakrhost',
       'peakghost', 'peakr_apphost', 'peakg_apphost', 'magnitude_change_after',
       'magnitude_change_before', 'panstarrs_r_mag', 'panstarrs_r_mag_err',
       'pipeline_ineffciency_host', 'pipeline_ineffciency_targetbyhost'],
      dtype='object') 330
Index(['name', 'firstdet', 'firstdetmag', 'lastnondet', 'texp', 'risetime',
       'risetimee', 'plateaustart', 'plateaustarte', 'plateauend',
       'plateauende', 'tailstart', 'tailstarte', 'tailstartmag',
       'plateaustartmag', 'plateauendmag', 'plateaumidmag', 'last_tail'],
      dtype='object') 140


In [15]:
df_merged = df_1.merge(df_2, on="name", how="inner").drop_duplicates(subset=['name'])
print(df_merged.columns, df_merged.shape)

Index(['name', 'firstdet_x', 'firstdetmag_x', 'jump', 'lastnondet_x', 'texp_x',
       'gap', 'Npre_g', 'Npre_r', 'Npost_g', 'Npost_r', 'slope_peak_r',
       'slope_peak_g', 'slope_peak_r10', 'slope_peak_g10', 'peakr', 'peakre',
       'peakg', 'peakge', 'peakrmjd', 'peakgmjd', 'redshift', 'peakr_app',
       'peakg_app', 'mean_EW', 'AV_NaD', 'colorexcess', 'Avhost', 'peakrhost',
       'peakghost', 'peakr_apphost', 'peakg_apphost', 'magnitude_change_after',
       'magnitude_change_before', 'panstarrs_r_mag', 'panstarrs_r_mag_err',
       'pipeline_ineffciency_host', 'pipeline_ineffciency_targetbyhost',
       'firstdet_y', 'firstdetmag_y', 'lastnondet_y', 'texp_y', 'risetime',
       'risetimee', 'plateaustart', 'plateaustarte', 'plateauend',
       'plateauende', 'tailstart', 'tailstarte', 'tailstartmag',
       'plateaustartmag', 'plateauendmag', 'plateaumidmag', 'last_tail'],
      dtype='object') (109, 55)


Never mind, there is no RA or DEC in the Zenodo sample.

In [16]:
# Borrowed from Viraj's LSST CCA Summer School module
# These credentials is publicly sourced from https://github.com/alercebroker/usecases/blob/master/alercereaduser_v4.json, in case
# you were (rightly) panicking about potential security breaches
params =  {
        "dbname" : "ztf",
        "user" : "alerceread",
        "host": "54.205.99.47",
        "password" : "w*C*u8AXZ4e%d+zv"
}

conn = psycopg.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

In [36]:
conn = psycopg.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

names = df_merged['name'].tolist()

names = df_merged['name'].tolist()

alerce = Alerce()
query_ended = False
counter = 1
all_alerts = pd.DataFrame()
while not query_ended:
    alerts = alerce.query_objects(
        format='pandas',
        oid=names,
        page=counter,
        page_size=10000,
    )
    if (len(alerts) == 0) or (counter > 10):
        query_ended = True
    all_alerts = pd.concat([all_alerts, alerts], ignore_index=True)
    counter += 1

In [40]:
print(all_alerts.columns, all_alerts.shape)

Index(['oid', 'ndethist', 'ncovhist', 'mjdstarthist', 'mjdendhist',
       'corrected', 'stellar', 'ndet', 'g_r_max', 'g_r_max_corr', 'g_r_mean',
       'g_r_mean_corr', 'firstmjd', 'lastmjd', 'deltajd', 'meanra', 'meandec',
       'sigmara', 'sigmadec', 'class', 'classifier', 'probability',
       'step_id_corr'],
      dtype='object') (108, 23)


In [38]:
all_alerts

,oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
0,ZTF18aaszvfn,93,2072,58248.310775,59391.288310,False,False,40,-0.063799,0.865591,...,59391.288310,83.960486,215.782862,50.220817,0.000044,0.000022,None,None,None,correction_0.0.1
1,ZTF18abjndhh,66,397,58324.166528,58486.489676,False,False,11,0.377247,NaN,...,58486.489676,162.323148,173.179280,62.432761,0.000074,0.000027,None,None,None,corr_bulk_0.0.1
2,ZTF18abzrgim,32,75,58388.269190,58494.133542,False,False,26,0.336168,NaN,...,58494.133542,105.864352,346.885593,23.005797,0.000030,0.000021,None,None,None,corr_bulk_0.0.1
3,ZTF19aadnxnl,482,1220,58502.489664,58901.430382,False,False,109,0.255573,NaN,...,58901.430382,398.940718,203.811186,44.766284,0.000041,0.000025,None,None,None,corr_bulk_0.0.1
4,ZTF19aailepg,83,377,58514.464630,58661.200752,False,False,66,0.216761,NaN,...,58661.200752,146.736123,169.467734,30.156520,0.000039,0.000040,None,None,None,corr_bulk_0.0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,ZTF24aafqzur,55,943,60355.520359,60471.292118,True,False,36,NaN,NaN,...,60471.292118,115.771759,216.488345,-2.392265,0.008317,0.008309,None,None,None,24.5.1
104,ZTF24aagupsf,183,3955,60374.513438,60473.367002,False,False,56,NaN,NaN,...,60473.367002,89.952558,247.672867,44.511143,0.010341,0.007374,None,None,None,24.5.1
105,ZTF24aajxppf,113,2120,60416.479329,60555.171528,True,False,81,NaN,NaN,...,60555.171528,136.799097,269.590144,9.681528,0.038562,0.038013,None,None,None,24.5.2a6
106,ZTF24aaplfjd,174,2001,60460.465995,60581.460706,True,False,63,NaN,NaN,...,60581.460706,120.994711,14.505587,30.706595,0.098861,0.085000,None,None,None,24.5.2a6


In [41]:
all_alerts[['oid', 'meanra', 'meandec']].to_csv('data/ZTF_coords.csv', index=False)